In [1]:
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
import numpy as np
from utils import listdir
import os

class PredictedVoxelDataset:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.list_category = sorted(os.listdir('predicted_EVC_conv5'))
        voxel_path = []
        categories = []
        nSamples = []
        for i, c in enumerate(self.list_category):
            voxel_path.extend([os.path.join(self.root_dir, c, s) for s in os.listdir(os.path.join(self.root_dir, c))])
            nSample = len([s for s in os.listdir(os.path.join(self.root_dir, c))])
            if i == 0:
                nSamples.append(nSample)
            else:
                nSamples.append(nSample+nSamples[i-1])
            for n in range(nSample):
                categories.append(i)
        self.voxel_path = voxel_path
        self.categories = categories
        self.nClass = len(set(categories))
        self.nSamples = nSamples
    def __len__(self):
        return len(self.voxel_path)
    def __getitem__(self, idx):
        #category = self.list_category[idx]
        voxels = np.load(self.voxel_path[idx])[0]
        #onehotclass = np.zeros(self.nClass)
        #onehotclass[self.categories[idx]] = 1
        sample = {'voxel': voxels, 'category': self.categories[idx]}
        return sample

In [2]:
voxel_dataset = PredictedVoxelDataset(root_dir='./predictedvggall/subj001/LOC_conv5')
print(len(voxel_dataset))
print(voxel_dataset[0]['voxel'].shape)
print(voxel_dataset[0]['category'])

voxels = np.stack(voxel_dataset[i]['voxel'] for i in range(len(voxel_dataset)))
categories = np.stack(voxel_dataset[i]['category'] for i in range(len(voxel_dataset)))


20692
(200,)
0


/home/chan21/anaconda3/envs/pytorch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
classifier = svm.SVC(kernel='linear')

fmri_score = np.zeros((voxel_dataset.nClass, voxel_dataset.nClass))
for i in range(0, voxel_dataset.nClass-2, 1):
    for j in range(i+2, voxel_dataset.nClass, 1):
        if i == 0:
            X_train, X_test, y_train, y_test = train_test_split(
             voxels[0:voxel_dataset.nSamples[j]], categories[0:voxel_dataset.nSamples[j]], test_size=0.5, shuffle=True)
        else:
            X_train, X_test, y_train, y_test = train_test_split(
             voxels[voxel_dataset.nSamples[i]:voxel_dataset.nSamples[j]], categories[voxel_dataset.nSamples[i]:voxel_dataset.nSamples[j]], test_size=0.5, shuffle=True)
        classifier.fit(X_train, y_train)
        predicted = classifier.predict(X_test)
        #print(metrics.classification_report(y_test, predicted))
        fmri_score[i,j-1] = classifier.score(X_test, y_test)

In [10]:
fmri_score

1470

In [5]:
voxel_dataset = PredictedVoxelDataset(root_dir='./predictedvggall_random/subj001/LOC_conv5')
print(len(voxel_dataset))
print(voxel_dataset[0]['voxel'].shape)
print(voxel_dataset[0]['category'])

voxels = np.stack(voxel_dataset[i]['voxel'] for i in range(len(voxel_dataset)))
categories = np.stack(voxel_dataset[i]['category'] for i in range(len(voxel_dataset)))

20692
(200,)
0


/home/chan21/anaconda3/envs/pytorch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
classifier = svm.SVC(kernel='linear')

random_score = []
for i in range(1, voxel_dataset.nClass-2, 1):
    X_train, X_test, y_train, y_test = train_test_split(
         voxels[voxel_dataset.nSamples[i]:voxel_dataset.nSamples[i+2]], categories[voxel_dataset.nSamples[i]:voxel_dataset.nSamples[i+2]], test_size=0.5, shuffle=True)
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    #print(metrics.classification_report(y_test, predicted))
    random_score.append(classifier.score(X_test, y_test))

np.mean(random_score)

0.7039373378934998

In [7]:
# disp = metrics.plot_confusion_matrix(classifier, X_test, y_test)
# disp.figure_.suptitle("Confusion Matrix")
# print("Confusion matrix:\n%s" % disp.confusion_matrix)